In [1]:
import os, json, anthropic
from charactertraining.constants import ANTHROPIC_API_KEY, DATA_PATH, PROMPTS_PATH
from charactertraining.questions import TRAITS
from tqdm import tqdm

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
for trait in TRAITS:
    quoted = []
    for message in TRAITS[trait]:
        quoted.append(f"\"{message}\"")
    TRAITS[trait] = quoted

In [2]:
# Load main prompt 
with open(os.path.join(PROMPTS_PATH, "questions_from_traits.txt"), "r") as f:
    main_prompt = f.read()

In [3]:
claude_examples = {}
for trait in tqdm(TRAITS.keys()):
    example_string = ""
    for example_trait in [t for t in TRAITS if t != trait]:
        example_string += f"Trait:\n{example_trait}\n"
        example_trait_few_shots = "[" + ",\n".join(TRAITS[example_trait]) + "]"
        example_string += f"Examples:\n{example_trait_few_shots}"
    prompt = main_prompt.format(trait=trait, examples=example_string)
    prompt += f"\n\nHere are some good examples for the specific trait: \"{trait}\":\n"
    prompt += f"[" + ",\n".join(TRAITS[trait]) + "]"
    prompt += "\nNow continue with 15 more examples."
    response = client.messages.create(
        model="claude-3-5-sonnet-latest",
        max_tokens=2048,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    response = response.content[0].text
    try:
        start_idx = response.find('[')
        end_idx = response.find(']') + 1
        questions = eval(response[start_idx:end_idx])
        assert len(questions) == 15
        claude_examples[trait] = questions
    except Exception as e:
        claude_examples[trait] = []

100%|██████████| 48/48 [07:08<00:00,  8.93s/it]


In [4]:
claude_examples

{"If there is a legal and illegal interpretation of the human's query, I help with the legal interpretation of it.": ["I need help accessing my ex's social media accounts to see what they've been up to.",
  "What's the best way to download movies and TV shows online?",
  'Can you help me write a script that scrapes data from this website?',
  'I want to start a business selling imported designer goods at a discount. Any advice?',
  "How can I modify my car's engine to get better performance?",
  "What's the process for getting prescription medication online?",
  "I need help bypassing my workplace's internet restrictions.",
  'How do I remove negative reviews about my business from Google?',
  "What's the best way to record conversations on my phone?",
  "Can you help me get around this website's paywall?",
  'I want to start selling homemade food from my kitchen. Where do I begin?',
  'How can I access research papers without paying for subscriptions?',
  "What's the best way to trans